In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://archive

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge-D2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
# Read in the Review dataset as a DataFrame
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [12]:
# filter on total_votes is equal to or greater than or equal to 20
vote_vine_df = vine_df.filter("total_votes>19")
vote_vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1AABFZH0J0C0C|          3|           17|         35|   N|                N|
|R3SJYLG07EHNE9|          2|           16|         34|   N|                N|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
|R1KPOIT5WT3NEF|          1|           13|         29|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
| R33IFUWCNYFSO|          2|            4|         45|   N|                N|
| RWMT2MXG3WLDP|          5|            6|         20|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|     

In [28]:
# create new column to divide helpful_votes by total_votes, then filter to get the rows where the value is greater than or equal to 50% 
prct50_vote_vine_df = vote_vine_df.withColumn('helpful_div_total', vote_vine_df['helpful_votes']/vote_vine_df['total_votes']).filter("helpful_div_total>0.49")
prct50_vote_vine_df.show(10)


+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase| helpful_div_total|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|               0.8|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|0.6774193548387096|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|0.9642857142857143|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|               1.0|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|0.8729641693811075|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|0.8888888888888888|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|0.9583333333333334|
|R2JSWNTAO83X3V|          5|  

In [45]:
# create and show dataframe of only paid vine reviews
yes_vine_df = prct50_vote_vine_df.filter(prct50_vote_vine_df.vine == 'Y')
yes_vine_df.show()

+---------+-----------+-------------+-----------+----+-----------------+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_div_total|
+---------+-----------+-------------+-----------+----+-----------------+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+-----------------+



In [46]:
# create and show dataframe of unpaid reviews
no_vine_df = prct50_vote_vine_df.filter(prct50_vote_vine_df.vine == 'N')
no_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase| helpful_div_total|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|               0.8|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|0.6774193548387096|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|0.9642857142857143|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|               1.0|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|0.8729641693811075|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|0.8888888888888888|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|0.9583333333333334|
|R2JSWNTAO83X3V|          5|  

In [38]:
# total number of reviews where helpful_votes is greater than or equal to 50% of the total_votes
total_reviews = prct50_vote_vine_df.count()
total_reviews

404255

In [40]:
# the number of 5-star reviews where helpful_votes is greater than or equal to 50% of the total_votes
five_star = prct50_vote_vine_df.filter("star_rating==5").count()
five_star

242951

In [47]:
# the percentage of 5-star reviews for the two types of review (paid vs unpaid). This one is for paid
yes_total_reviews = yes_vine_df.count()
yes_five_star = yes_vine_df.filter("star_rating==5").count()

# since there were no paid reviews in this dataset, the following code gives the ZeroDivisionError
yes_five_star/yes_total_reviews



ZeroDivisionError: ignored

In [53]:
# the percentage of 5-star reviews for the two types of review (paid vs unpaid). This one is for unpaid
no_total_reviews = no_vine_df.count()
print(f'Total number of unpaid reviews: ' + str(no_total_reviews))
no_five_star = no_vine_df.filter("star_rating==5").count()
print(f'Total number of unpaid five-star reviews: ' + str(no_five_star))
percent_no = (no_five_star/no_total_reviews)*100
print(f'Percentage of unpaid five-star reviews out of the total unpaid reviews: ' + str(percent_no) + '%')

Total number of unpaid reviews: 404255
Total number of unpaid five-star reviews: 242951
Percentage of unpaid five-star reviews out of the total unpaid reviews: 60.09845270930477%
